In [5]:
import numpy as np
import pandas as pd
import os
import scipy.io
import matplotlib.pyplot as plt
from scipy import signal

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
PathCsv = "/content/drive/MyDrive/CSVsb4filterADHDAndNon/dfNonADHDhalf.csv"
dfNonADHDnew=pd.read_csv(PathCsv)

In [8]:
len(dfNonADHDnew)

7999921

In [9]:
dfNonADHDnew.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,Label
0,0,-407.0,-19.0,-425.0,-19.0,-388.0,-20.0,-204.0,200.0,-351.0,...,2923.0,163.0,420.0,126.0,347.0,126.0,121.0,121.0,156.0,Non_ADHD
1,1,-125.0,261.0,-20.0,261.0,-204.0,273.0,-241.0,236.0,-204.0,...,-131.0,200.0,-278.0,236.0,-20.0,163.0,-19.0,402.0,-160.0,Non_ADHD
2,2,-19.0,437.0,16.0,367.0,-94.0,384.0,-278.0,273.0,-57.0,...,16.0,420.0,-131.0,384.0,16.0,384.0,50.0,578.0,-125.0,Non_ADHD
3,3,-301.0,-125.0,-315.0,-55.0,-388.0,-20.0,-241.0,163.0,-94.0,...,-351.0,-351.0,-388.0,16.0,-131.0,347.0,-442.0,85.0,-231.0,Non_ADHD
4,4,-336.0,-301.0,-278.0,-90.0,-388.0,-20.0,-241.0,310.0,163.0,...,-499.0,-241.0,-462.0,163.0,52.0,384.0,-336.0,-55.0,-336.0,Non_ADHD


In [10]:
dfNonADHDnew.shape

(7999921, 21)

In [11]:
channel_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8','P7','P8','Fz','Cz','Pz']
dfNonADHDnew.drop(columns='Unnamed: 0', axis=1, inplace=True)


In [12]:
dfNonADHDnew.shape


(7999921, 20)

In [13]:
dfNonADHDnew.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Label
0,-407.0,-19.0,-425.0,-19.0,-388.0,-20.0,-204.0,200.0,-351.0,200.0,2923.0,163.0,420.0,126.0,347.0,126.0,121.0,121.0,156.0,Non_ADHD
1,-125.0,261.0,-20.0,261.0,-204.0,273.0,-241.0,236.0,-204.0,236.0,-131.0,200.0,-278.0,236.0,-20.0,163.0,-19.0,402.0,-160.0,Non_ADHD
2,-19.0,437.0,16.0,367.0,-94.0,384.0,-278.0,273.0,-57.0,347.0,16.0,420.0,-131.0,384.0,16.0,384.0,50.0,578.0,-125.0,Non_ADHD
3,-301.0,-125.0,-315.0,-55.0,-388.0,-20.0,-241.0,163.0,-94.0,200.0,-351.0,-351.0,-388.0,16.0,-131.0,347.0,-442.0,85.0,-231.0,Non_ADHD
4,-336.0,-301.0,-278.0,-90.0,-388.0,-20.0,-241.0,310.0,163.0,273.0,-499.0,-241.0,-462.0,163.0,52.0,384.0,-336.0,-55.0,-336.0,Non_ADHD


In [14]:
dfNonADHDnew = dfNonADHDnew.drop(columns='Label')

In [18]:
#Need to install pip install mne to run next cell 
# pip install mne

In [19]:
# # Data Visualisation ADHD: 
# import numpy as np
# import pandas as pd 
# #Valid code, commented out for running time.

# import mne

# path = PATH

# sfreq = 128 
# info = mne.create_info(ch_names = channel_names, sfreq = sfreq)
# raw = mne.io.RawArray(dfAll.transpose(), info)
# raw.plot()

In [20]:
#After visual inspection: Not dropping any column as visual inspection does not show bad channels. The fluctuations can be corrected in filtering. 

# Filtering using scipy

In [21]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import csv
import pickle
import os
import seaborn as sns
from scipy.signal import butter, filtfilt
from scipy.stats import zscore
import scipy.io

import matplotlib.pyplot as plt
import io
import matplotlib.cm as cm
from collections import defaultdict

In [22]:
#Filtering ADHD
def filter(data, lowFreq, highFrequency, sf):
    data_filter = np.zeros_like(dfNonADHDnew)
    f_Nyquist = sf / 2.  # Nyquist frequency
    b_low = lowFreq / f_Nyquist  # normalized frequency [0..1]
    b_high = highFrequency / f_Nyquist  # normalized frequency [0..1]
    # band-pass filter parameters
    p_lowp = {"N":6, "Wn":b_high, "btype":"lowpass", "analog":False, "output":"ba"}
    p_highp = {"N":6, "Wn":b_low, "btype":"highpass", "analog":False, "output":"ba"}
    bp_b1, bp_a1 = butter(**p_lowp)
    bp_b2, bp_a2 = butter(**p_highp)
    data_filter = filtfilt(bp_b1, bp_a1, data, axis=0)
    data_filter = filtfilt(bp_b2, bp_a2, data_filter, axis=0)
    return data_filter


In [23]:

# #  Function to read in the EEG data and extract the valid lead data, low and high pass filter and z-transform the data.
# #  Returns a dataframe.
def get_EEG_data(path, filename):
    hz = 128
    dat = dfNonADHDnew.filter(list(range(0, 18)))
    dat.columns = list(range(1, 15))
    dat = dat.filter([0,1,2, 3, 4,5,6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19], axis=1)
    labels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8','P7','P8','Fz','Cz','Pz']
    dat.columns = labels

      # Filter the data, high pass 2 Hz, low pass 40 Hz.
    lo, hi = 2, 40
    # Do the filtering.
    datf = filter(dat.to_numpy(), lo, hi, hz)

    # Convert back to a dataframe.
    dat = pd.DataFrame({c: datf[:, i] for i,c in enumerate(labels)})

    # Z-transform each column
    #dat = dat.apply(zscore)
    
    return dat


In [24]:

# Filter the data, high pass 2 Hz, low pass 40 Hz.
lo, hi = 2, 40
hz = 128
# Do the filtering.
datf = filter(dfNonADHDnew.to_numpy(), lo, hi, hz)

In [16]:
datf.shape

(7999921, 19)

In [17]:
dfilter = pd.DataFrame(datf)
dfilter.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-1.992904,-12.470122,5.663157,-12.395136,5.075235,-12.908046,16.865728,-14.084541,14.040510,-12.325555,-1.222646,-11.717585,7.841180,-13.853751,-5.212017,-19.386475,-3.462614,-55.132219,28.937536
1,304.243883,341.762906,425.326442,315.654157,230.093212,322.491051,-20.802611,58.323170,176.192500,67.393095,-2532.763927,165.042046,-527.572262,165.487069,-278.981919,49.185659,-38.856991,301.618389,-175.094539
2,308.241137,313.549322,384.379138,283.010995,216.131644,298.125867,-34.613615,23.094911,238.688409,79.749385,-2914.988718,42.531836,-604.776956,139.095169,-368.550537,176.261403,-169.192180,274.462460,-278.520803
3,134.483346,-61.184575,136.028345,0.343378,47.971586,38.326598,-36.064091,5.062730,307.911751,56.321237,-2455.719831,-338.886343,-579.900981,-10.447265,-307.271187,251.045623,-364.826421,-33.582477,-316.610281
4,-7.761324,-351.658432,72.475985,-138.637483,-53.859716,-87.038544,11.284288,101.498660,429.689043,37.404065,-2638.598433,-519.536338,-710.243830,-49.845464,-230.214455,181.116066,-494.663121,-254.182918,-226.986040


In [19]:
dfilter.shape

(7999921, 19)

In [21]:
dfilter.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-0.011934,-0.075730,0.038482,-0.089194,0.034696,-0.093457,0.110020,-0.100557,0.088101,-0.087849,-0.008160,-0.080745,0.057005,-0.106624,-0.036856,-0.156298,-0.022972,-0.394090,0.199270
1,1.821903,2.075446,2.890196,2.271382,1.572959,2.334867,-0.135697,0.416403,1.105563,0.480334,-16.940626,1.137286,-3.835032,1.273643,-1.972928,0.396542,-0.257804,2.155985,-1.205713
2,1.845839,1.904111,2.611949,2.036489,1.477515,2.158460,-0.225789,0.164888,1.497709,0.568402,-19.497174,0.293081,-4.396250,1.070522,-2.606349,1.421048,-1.122542,1.961873,-1.917916
3,0.805326,-0.371561,0.924345,0.002469,0.327943,0.277487,-0.235250,0.036146,1.932068,0.401421,-16.425309,-2.335230,-4.215421,-0.080407,-2.172987,2.023971,-2.420522,-0.240051,-2.180203
4,-0.046477,-2.135542,0.492492,-0.997609,-0.368194,-0.630169,0.073612,0.724657,2.696189,0.266592,-17.648511,-3.580069,-5.162912,-0.383628,-1.628050,1.460187,-3.281953,-1.816916,-1.563043


In [20]:
labels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8','P7','P8','Fz','Cz','Pz']
dfilter.columns = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8','P7','P8','Fz','Cz','Pz']

In [21]:
#IGNORE THIS CELL. NOT USED. For future implementation only.

PATHEEGSignalPlot="/content/drive/MyDrive/EEGSignalPlot/"
def plotleads(dat, leads, start, seconds, hz,state):
    for i, lead in enumerate(leads):   
        #print('Lead ' + lead)
        a=PATHEEGSignalPlot+'Lead_'+str(lead)+'_'+str(state)+".png"
        plt.figure(figsize=(16,10))
        ax = plt.gca()
        k=(start+seconds*hz)
        print(str(start) + ':'+ str(k))
        plt.plot(dat.loc[start: k, lead])   
        plt.title('Lead ' + lead + state)
        plt.savefig(a)
        plt.show()
        _ = ax.axis('off')

In [23]:
dfilter.head() #filtered data only for ADHD

,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz
0,-1.992904,-12.470122,5.663157,-12.395136,5.075235,-12.908046,16.865728,-14.084541,14.040510,-12.325555,-1.222646,-11.717585,7.841180,-13.853751,-5.212017,-19.386475,-3.462614,-55.132219,28.937536
1,304.243883,341.762906,425.326442,315.654157,230.093212,322.491051,-20.802611,58.323170,176.192500,67.393095,-2532.763927,165.042046,-527.572262,165.487069,-278.981919,49.185659,-38.856991,301.618389,-175.094539
2,308.241137,313.549322,384.379138,283.010995,216.131644,298.125867,-34.613615,23.094911,238.688409,79.749385,-2914.988718,42.531836,-604.776956,139.095169,-368.550537,176.261403,-169.192180,274.462460,-278.520803
3,134.483346,-61.184575,136.028345,0.343378,47.971586,38.326598,-36.064091,5.062730,307.911751,56.321237,-2455.719831,-338.886343,-579.900981,-10.447265,-307.271187,251.045623,-364.826421,-33.582477,-316.610281
4,-7.761324,-351.658432,72.475985,-138.637483,-53.859716,-87.038544,11.284288,101.498660,429.689043,37.404065,-2638.598433,-519.536338,-710.243830,-49.845464,-230.214455,181.116066,-494.663121,-254.182918,-226.986040


In [24]:
dfilter.to_csv('/content/drive/MyDrive/CSVsb4filterADHDAndNon/ADHDfiltered.csv')

#Too much data so colab may crash. So import again dataframe (Uncomment below)- pandas as pd pd.read_csv


In [25]:
# X = pd.read_csv('/content/drive/MyDrive/CSVsb4filterADHDAndNon/ADHDfiltered.csv')

##Apply ICA to filtered data

In [26]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA
X = dfilter
transformer = FastICA(n_components=7,random_state=0,whiten='unit-variance')
X_transformed = transformer.fit_transform(X) #Had worked with df at beginning.
X_transformed.shape
# (1797, 7)
print(X_transformed[0:5, :])
print(X_transformed.shape)

[[-2.19945747e-05  8.22944916e-05 -8.26601417e-05 -1.78250286e-05
  -3.51873117e-05 -2.98978625e-05 -8.36746322e-05]
 [-4.43552813e-03  1.12160383e-03  1.53517260e-03 -7.43729172e-04
   1.50172329e-03  9.06664055e-04  4.11655548e-04]
 [-4.85325507e-03  1.03230041e-03  1.91592625e-03 -7.20818497e-04
   1.77760468e-03  7.36348614e-04  4.41306983e-04]
 [-3.52742117e-03  6.83509431e-04  1.81822786e-03 -2.57205267e-04
   1.50546502e-03 -1.29031117e-04  3.56511553e-05]
 [-3.67399245e-03  1.12296548e-03  1.68920782e-03  7.52902873e-06
   1.53960692e-03 -5.73350870e-04 -2.94665421e-04]]
(7999921, 7)


In [27]:
dfICA_Non = pd.DataFrame(X_transformed)

In [28]:
dfICA_Non.head()


,0,1,2,3,4,5,6
0,-0.000022,0.000082,-0.000083,-0.000018,-0.000035,-0.000030,-0.000084
1,-0.004436,0.001122,0.001535,-0.000744,0.001502,0.000907,0.000412
2,-0.004853,0.001032,0.001916,-0.000721,0.001778,0.000736,0.000441
3,-0.003527,0.000684,0.001818,-0.000257,0.001505,-0.000129,0.000036
4,-0.003674,0.001123,0.001689,0.000008,0.001540,-0.000573,-0.000295


In [29]:
len(dfICA_Non)

7999921

In [30]:
dfICA_Non.columns = ['A','B','C','D','E','F','G']
dfICA_Non.head()
dfICA_Non['Label'] = pd.Series(['ADHD' for x in range(len(dfICA_Non.index))])


In [31]:
dfICA_Non.head()

,A,B,C,D,E,F,G,Label
0,-0.000022,0.000082,-0.000083,-0.000018,-0.000035,-0.000030,-0.000084,ADHD
1,-0.004436,0.001122,0.001535,-0.000744,0.001502,0.000907,0.000412,ADHD
2,-0.004853,0.001032,0.001916,-0.000721,0.001778,0.000736,0.000441,ADHD
3,-0.003527,0.000684,0.001818,-0.000257,0.001505,-0.000129,0.000036,ADHD
4,-0.003674,0.001123,0.001689,0.000008,0.001540,-0.000573,-0.000295,ADHD


In [32]:
dfICA_Non.to_csv('/content/drive/MyDrive/CSVsb4filterADHDAndNon/dfICA_Dataset1NonADHD.csv')
